# Linear Regression

Download the required libraries for this notebook by running the cell below. If you are using conda, uncomment and run the conda command. If you are using PIP, uncomment and use the PIP command.

COME BACK AND MAKE CONDA USE ENVIRONMENT.YML TO LOAD INSTEAD OF REQUIREMENTS.TXT

In [2]:
import sys
#!conda install --yes --prefix {sys.prefix} 
#!{sys.executable} -m pip install -r requirements.txt

In [4]:
import numpy as np
import pandas as pd
from scipy import stats

## Import the Data

COME BACK AND ADD OWN HOSTED DATA TO PLAY WITH

In [10]:
data = pd.read_csv('https://raw.githubusercontent.com/maalvarezl/MLAI/master/Labs/datasets/olympic_marathon_men.csv', header=None, encoding= 'unicode_escape')
x = np.array(data.iloc[:, 0].values).reshape(-1,1)
y = np.array(data.iloc[:, 1].values).reshape(-1,1)

# Print the shapes of the data for x and y
print(f'X: {x.shape}\nY: {y.shape}')

X: (27, 1)
Y: (27, 1)


## Extension: The Maths
If you are interested in the maths of linear regression, this section will cover how to implement linear regression manually in numpy. This will also cover the underlying linear algebra involved